In [ ]:
# Stock Price Indicator
                
This project aims to predict adjusted close price of some stocks chosen previously.


In [ ]:
##1- Data Exploration


In [23]:
import pandas as pd
print(pd.__version__)

CSCO = pd.read_csv('data/CSCO.csv', header=None)
CSCO.head()


0.24.2


,0,1,2,3,4,5,6
0,Date,Open,High,Low,Close,Adj Close,Volume
1,2019-01-28,45.730000,45.770000,45.230000,45.750000,45.750000,18296700
2,2019-01-29,45.810001,46.049999,45.650002,45.959999,45.959999,14470100
3,2019-01-30,45.939999,46.930000,45.919998,46.709999,46.709999,18329300
4,2019-01-31,46.720001,47.380001,46.520000,47.290001,47.290001,23693600


In [20]:
from pandas_datareader import get_data_yahoo
 
import matplotlib.pyplot as plt

data = get_data_yahoo("FITB", start = '2012-01-01', end = '2012-12-31')[['Close', 'Volume']]

data.plot(subplots = True, figsize = (8, 8));

plt.legend(loc = 'best')
plt.show()


ModuleNotFoundError: No module named 'pandas_datareader'